 # **Music genre classification**
 
License : CC BY-NC-SA 4.0

This notebook enables training a Deep Learning model.

The purpose is to train a model to automatically classify music according to its genre.

For this, we will use a database (GTZAN) which will be used to train several models. 

We will use the PyTorch framework and its torchaudio/torchvision extensions. 

Computations will be hardware-accelerated by the use of Jean-Zay GPUs.

> **Let's check the loaded modules and the presence of the reserved GPU(s): the notebook needs to be launched from a compute node and not a front-end to take advantage of the GPUs.**


> **The "!nvidia-smi" command enables quickly visualizing the status of the GPUs.**

In [ ]:
import torch
print(torch.__version__)
import torchaudio

In [ ]:
!module list
!conda env list
!nvidia-smi
from torch.cuda import device_count
print('Torch detecting {0} GPUs compatible with CUDA'.format(device_count()))

## **Imports**

Multiple libraries will be used throughout the course to carry out data transformations, file management, visualization, etc.

> **We use the pytorch-gpu/py3/1.11.0 environment which contains all the libraries useful for this project.**

> **You can modify the imported scripts and re-import them without restarting the kernel thanks to the autoreload command.**

In [ ]:
#!module load pytorch-gpu/py3/1.9.0
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np # Maths
import pandas as pd #Data analysis and manipulation
#from sklearn.utils import shuffle

import os # Operating System interaction

import matplotlib # Visualization
import matplotlib.pyplot as plt
#Necessary to view the figures in the notebook
%matplotlib inline 

from tqdm import tqdm #progress bar

import torch

> **Warning: Windows users must install soundfile (pip install soundfile) and add the following line :**

> torchaudio.set_audio_backend("sox_io")

In [ ]:
# Set randomness for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Some variables for file management
project_dir = os.getcwd()
data_dir = '/mnt/datasets/TP_Methodologie/GTZAN/data'
figures_dir = os.path.join(project_dir, 'figures')
scripts_dir = os.path.join(project_dir, 'scripts')
models_dir = os.path.join(project_dir, 'models')
dataset_dir = os.path.join(data_dir, 'raw')

---
---
---
---
---
## **Dataset splitting**


To acquire a model capable of determining the genre of a musical excerpt, it is necessary to separate the database into several parts that do not share common extracts in order to avoid training a model that is too specialized. The result of the evaluation will be biased if information leaks from the training dataset to the test dataset or if these data are too correlated.

Once satisfied with a model, we can then consider using the whole dataset to improve performance even further or to at least reduce the portion dedicated to the evaluation.

> We can use the standard split proposed by torchaudio which can be useful to easily compare our performance with others around a standardised task. We can also define a manual split facilitated by the torch.utils.data.Dataset and torch.utils.data.random_split classes.

> In this exercise, the valid set is used to evaluate whether the loss of the model is decreasing correctly. In the opposite case, we stop the training (practice called early stopping).

> **Do not hesitate to modify the separation of the data for your different training sessions. To do this, use the "train_ratio", "valid_ratio" and "test_ratio " parameters.**

> Reminder: Training and test data should not be shared.

In [ ]:
from torch import nn
from scripts.utils import load_gtzan_sets, crop_audio
import torchaudio.transforms as tat
import torchvision.transforms as tvt
transformation = tvt.Compose([tvt.Lambda(crop_audio)])
train_set, valid_set, test_set = load_gtzan_sets(root = dataset_dir, filtered = True, generator = 2022, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, transform = transformation)
print(' Train size : {} \n Valid size : {} \n Test_size : {}'.format(len(train_set), len(valid_set), len(test_set)))

---
---
---
---
---
## **Transformation and Data Augmentation**

It is possible to perform transformations on data to emphasize certain features, to match the data with those for which a model has shown the best performance or to improve training.

Data augmentation is often recommended to make the model more robust to certain effects that are rare in the dataset but can be generated artificially.

In the case of music, although images have been generated, the exercise remains very different from the classification of animal photos. Indeed, orientation changes and geometric deformations are meaningless for a spectrogram.

All the transformations are grouped in a module (tvt.Compose) which, when a data item is called, applies the sequence of transformations to the data item.

The transformations are not done upstream when the dataset is declared but when the data is called. Thus each epoch is different provided there is a probability of whether or not a transformation will be applied and a different seed at each epoch. This is how each epoch makes use of different data and how the amount of available data is artificially increased.

> **Not all transformations are useful. It is of course possible to try them, but thinking about the nature of the data beforehand is usually more effective.**

> **Be aware that if data preparation is to be applied to both training and test data sets, augmentation should only be used for training. Define two different transformations depending on the set concerned, taking inspiration from what was explored at the end of the first notebook.





In [ ]:
from torch import nn
from scripts.utils import load_gtzan_sets, crop_audio
import torchaudio.transforms as tat
import torchvision.transforms as tvt
from scripts.utils import check_sizes
transformation = tvt.Compose([tvt.Lambda(lambda waveform: crop_audio(waveform, duration = 5, augment=True)),
                              tat.Spectrogram()])
test_transformation = tvt.Compose([tvt.Lambda(lambda waveform: crop_audio(waveform, duration = 5, augment=False)),
                                  tat.Spectrogram()])

train_set, _, _ = load_gtzan_sets(root = dataset_dir, filtered = True, generator = 2022, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, transform = transformation)
_, valid_set, test_set = load_gtzan_sets(root = dataset_dir, filtered = True, generator = 2022, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, transform = test_transformation)

sizes_uniformity, sizes = check_sizes(train_set)

> Make sure you have all the output with the same dimensions for training from scratch or with the dimensions of the data used for the initial training for transfer learning.

> It is sometimes necessary to transform the data to match an expected input format of a model (number of channels in an image, for example). In this tutorial, we focus on modifying the model.


---
---
---
---
---
## **Data Loader**

> The DataLoader is a utility class which enables browsing databases (datasets) according to a predefined strategy: batch size, sampling, randomization, ...
Depending on the strategy chosen, the training of a model can vary greatly.

> **During the training, it is necessary to make compromises between batch size and memory occupation, while monitoring the convergence of our model towards a satisfactory solution. During your trainings, if you have memory overflows or if you notice that there is some memory remaining, you can modify the batch_size parameter. It may be necessary to restart the kernel.**

> **Reducing the batch size during the training can have a regularization effect.**


In [ ]:
from torch.utils.data import DataLoader

batch_size = 10
train_loader = DataLoader(train_set, batch_size, shuffle=True, num_workers=8, pin_memory=False)
valid_loader = DataLoader(valid_set, 1, num_workers=4, pin_memory=False)
test_loader = DataLoader(test_set, 1, num_workers=4, pin_memory=False)

---
---
---
---
---
## **Model architecture definition**

The model architectures prepared for this exercise are all available in 'scripts/models.py' or in torchvision.models: https://pytorch.org/vision/stable/models.html

> **You can try changing the model to compare performance. You will probably have to change the number of classes and the first layer of the model which expects an RGB image (3 channels). By visualizing the structure of the model, you should be able to deduce how to modify the first layer to fit. In the comments, you will find the necessary modifications for VGG and Resnet.**

> **In order to take advantage of the hardware acceleration provided by the GPU, it is necessary to move the model into the GPU memory. Set the device to CPU to observe the difference in speed compared to a GPU.**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Currently used device is :', device)

In [ ]:
from scripts.models import *
import torchvision.models as models

model = models.vgg16(num_classes=10)
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

#model = models.resnet18(num_classes=10)
#model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

model.to(device)

> If multiple GPUs are available, parallelization strategies can be used to speed up the training. Several parallelization strategies can be used. The easiest to implement is Data Parallelism: http://www.idris.fr/jean-zay/gpu/jean-zay-gpu-torch-multi.html

---
---
---
---
---
## **Model training**

> **In addition to the model, it is also possible to modify the training parameters of the model (hyper-parameters): optimizer, learning rate, batch size, number of epochs, etc.**

> The choice of optimizer has an impact on the convergence of the model. **Several optimizers are available: https://pytorch.org/docs/1.11/optim.html**

> You should not have to change the loss evaluation criterion although other criteria exist for the classification.

In [ ]:
start_epoch = 1
nbr_epochs = 50
epoch_interval = 1
batch_interval = 10
verbose = True
history = None

### **When restarting a training, do so from the cell below. If you change the model and data, do so from the cell above.**

In [ ]:
from torch import optim
lr = 0.001
momentum = 0.9
weight_decay = 0.0001
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

> **You can interrupt the training at nearly any time by using the stop button on the jupyterlab. The outputs will still be returned.**

> **This allows you to adjust the hyper-parameters during the training. Be careful not to reset the epoch and history counter.**

In [ ]:
import time
from scripts.utils import train
experiment_name = str(type(model))[:-2].split('.')[-1] + '_' +str(int(time.time()))


model, history, start_epoch = train(train_loader, 
                                valid_loader, 
                                model,
                                optimizer, 
                                criterion,
                                start_epoch,
                                nbr_epochs,
                                device, 
                                epoch_interval,
                                batch_interval,
                                verbose,
                                models_dir,
                                experiment_name,
                                history)

---
---
---
---
---
## **Model evaluation**

The final evaluation of the model should be done on data which has not been used for training this model.
We evaluate the best performance of the model. For this purpose, during each training session, the model was automatically saved in the models folder when a performance gain was observed.
> **Go to the models folder and copy the name of the model you wish to evaluate into the cell below.**

In [ ]:
from scripts.utils import test
#model_name = ".t7" 
#model_path = os.path.join(models_dir, model_name)
#model = torch.load(model_path)
test_loss, predictions, targets = test(test_loader, model, criterion, device)

### **Cost function evolution**

By observing the loss, we can determine the quality of learning.
We can deduce :
- A convergence that is too slow, requiring an increase in the learning rate/momentum
- Instability which, conversely, may require a reduction in the learning rate
- Overfitting
- Underfitting
- Differences too large between datasets, which does not allow the information learned from the training dataset to be generalized to the validation dataset
- Poor representativeness of the evaluation dataset

In [ ]:
from scripts.utils import plot_loss
plot_loss(history['loss'],history['val_loss'])

### **Accuracy evolution**


The success rate called accuracy provides direct information to assess the achievement of the desired task.

However, it is rarely self-sufficient. For example, it does not show classification biases related to disparities between classes.

> Other metrics exist, and choosing the right one is essential to avoid disappointment when using the model on new data.

In [ ]:
from scripts.utils import plot_accuracy
plot_accuracy(history['acc'],history['val_acc'])

### **The confusion matrix**

There is also the confusion matrix which, when used for classification tasks enables the quality of the classifier to be determined relative to each class.

> **It is preferable to try to normalize the metrics in order to have an absolute scale. For example, the confusion matrix can be normalized to the number of elements per class.**

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

gtzan_genres = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]
cm = sklearn.metrics.confusion_matrix(targets, predictions, normalize='true')
df_cm = pd.DataFrame(cm, index = [i for i in gtzan_genres],
                  columns = [i for i in gtzan_genres])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

---
---
---
---
---
## **Saving the model**

> **Do not forget to save the model to a file. You do not want to redo the whole same training. It is also good practice to save the model while you are training it so that you can resume from a saved point.**

In [ ]:
model_type = 
model_size =
nbr_epochs =
extra_info =
model_name = model_type + '_' + model_size + '_ '+ nbr_epochs + '_' + extra_info
torch.save(model, os.path.join(models_dir,model_name))

---
---
---
---
---